In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Step 1: Data Exploration and Preprocessing
df = pd.read_csv(r'C:\Users\User\Downloads\Neural networks\Neural networks\Alphabets_data.csv')
print(df.head())

# Separate features (X) and target (y)
X = df.drop('letter', axis=1)
y = df['letter']

# Convert categorical labels to numeric
y = pd.factorize(y)[0]

# Normalize the feature data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 2: Define a function to create and train the ANN model
def create_ann(hidden_units=64, learning_rate=0.001, epochs=10, batch_size=32):
    model = Sequential()
    model.add(Dense(hidden_units, input_dim=X_train.shape[1], activation='relu'))  # Input layer
    model.add(Dense(32, activation='relu'))  # Hidden layer
    model.add(Dense(len(np.unique(y)), activation='softmax'))  # Output layer (multi-class)
    
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Early stopping callback to prevent overfitting and speed up training
    early_stopping = EarlyStopping(monitor='loss', patience=3, verbose=1, restore_best_weights=True)
    
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0, callbacks=[early_stopping])
    return model

# Step 3: Reduce Hyperparameter Search Space (smaller search space and fewer epochs)
hidden_units_list = [32, 64]
learning_rate_list = [0.001]
epochs_list = [10]  # Reduced number of epochs to speed up the process
batch_size_list = [32]

best_model = None
best_score = 0
best_params = {}

# Loop through hyperparameters
for hidden_units in hidden_units_list:
    for learning_rate in learning_rate_list:
        for epochs in epochs_list:
            for batch_size in batch_size_list:
                print(f"Training model with hidden_units={hidden_units}, learning_rate={learning_rate}, epochs={epochs}, batch_size={batch_size}")
                
                # Create and train the model
                model = create_ann(hidden_units=hidden_units, learning_rate=learning_rate, epochs=epochs, batch_size=batch_size)
                
                # Evaluate the model
                y_pred = np.argmax(model.predict(X_test), axis=-1)
                accuracy = accuracy_score(y_test, y_pred)
                
                print(f"Accuracy: {accuracy:.4f}")
                
                # Save the best model based on accuracy
                if accuracy > best_score:
                    best_score = accuracy
                    best_model = model
                    best_params = {
                        'hidden_units': hidden_units,
                        'learning_rate': learning_rate,
                        'epochs': epochs,
                        'batch_size': batch_size
                    }

# Step 4: Final Evaluation of the Best Model
y_pred = np.argmax(best_model.predict(X_test), axis=-1)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Best Hyperparameters: {best_params}")
print(f"Final Model Evaluation:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


  letter  xbox  ybox  width  height  onpix  xbar  ybar  x2bar  y2bar  xybar  \
0      T     2     8      3       5      1     8    13      0      6      6   
1      I     5    12      3       7      2    10     5      5      4     13   
2      D     4    11      6       8      6    10     6      2      6     10   
3      N     7    11      6       6      3     5     9      4      6      4   
4      G     2     1      3       1      1     8     6      6      6      6   

   x2ybar  xy2bar  xedge  xedgey  yedge  yedgex  
0      10       8      0       8      0       8  
1       3       9      2       8      4      10  
2       3       7      3       7      3       9  
3       4      10      6      10      2       8  
4       5       9      1       7      5      10  
Training model with hidden_units=32, learning_rate=0.001, epochs=10, batch_size=32


D:\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Restoring model weights from the end of the best epoch: 10.
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Accuracy: 0.8670
Training model with hidden_units=64, learning_rate=0.001, epochs=10, batch_size=32


D:\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Restoring model weights from the end of the best epoch: 10.
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.8950
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Best Hyperparameters: {'hidden_units': 64, 'learning_rate': 0.001, 'epochs': 10, 'batch_size': 32}
Final Model Evaluation:
Accuracy: 0.8950
Precision: 0.8982
Recall: 0.8950
F1 Score: 0.8955
